<a href="https://colab.research.google.com/github/Location-Artistry/GEO-DEV-NOTEBOOKS/blob/main/GOOGLE_DOCS_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Colab libraries install & mount google drive to access credentials
#!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib 
import os
from __future__ import print_function
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir('/content/drive/MyDrive/CODE')

In [ ]:
# Get and return GoogleAPIS credentials
def getGcreds(CLIENT_SECRETS_FILE, SCOPES):
  flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
  credentials = flow.run_console()
  return credentials

# Create copy of Gdoc from template
def copyFile(DOCUMENT_ID, copyTitle, credentials):
  API_SERVICE_NAME = 'drive'
  API_VERSION = 'v3'
  service = build(API_SERVICE_NAME, API_VERSION, credentials = credentials)
  newText = {'name': copyTitle}
  try:
    result = service.files().copy(fileId=DOCUMENT_ID, body=newText).execute()
    display(result)
    return result['id']
  except (errors.HttpError, error):
    print('An error occurred: %s') % error
  return None

# List files in drive, pass number desired file results
def listFiles(num, credentials):
  API_SERVICE_NAME = 'drive'
  API_VERSION = 'v3'
  service = build(API_SERVICE_NAME, API_VERSION, credentials = credentials)
  results = service.files().list(
        pageSize=num, fields="nextPageToken, files(id, name, mimeType, size, parents, modifiedTime)").execute()
  items = results['files']
  # list all 20 files & folders
  for x, i in enumerate(items):
    display(f'{x} - {i.get("id")} - {i.get("mimeType")} - {i.get("parents")} - {i.get("modifiedTime")}')
  #for item in items:
  #  display(item)
  return items

  # Drive search function, template from Google Developers
def fileSearch(type,query,credentials):
  f = {'jpeg': "mimeType='image/jpeg'",'pdf':"mimeType='application/pdf'",'doc':"mimeType='application/vnd.google-apps.document'",
     'sheet':"mimeType='application/vnd.google-apps.spreadsheet'",'folder':"mimeType='application/vnd.google-apps.folder'",'png':"mimeType='image/png'",
     'text':"mimeType='text/plain'"}
  API_SERVICE_NAME, API_VERSION, page_token = 'drive','v3',None
  drive_service = build(API_SERVICE_NAME, API_VERSION, credentials = credentials)
  query = (f"{f[type]}{query}")
  while True:
    response = drive_service.files().list(q=query,spaces='drive',fields='nextPageToken, files(id, name)',pageToken=page_token).execute()
    for x, file in enumerate(response.get('files', [])):
      display(f'Found {x}: {file.get("name")} - {file.get("id")}')
    page_token = response.get('nextPageToken', None)
    if page_token is None:
      break
  return response['files']
  
# Update document with spcified requests
def updateDoc(DOCUMENT_ID, req, credentials):
  API_SERVICE_NAME = 'docs'
  API_VERSION = 'v1'
  service = build(API_SERVICE_NAME, API_VERSION, credentials = credentials)
  result = service.documents().batchUpdate(documentId=DOCUMENT_ID, body={'requests': req}).execute()
  display(f'Edit Successful for id: {result["documentId"]} Changes made: {result["replies"][0]}')
  return result
# Get Gdoc item information for inserting at index location
def getDoc(DOCUMENT_ID, req, credentials):
  try:
    API_SERVICE_NAME = 'docs'
    API_VERSION = 'v1'
    service = build(API_SERVICE_NAME, API_VERSION, credentials = credentials)
    result = service.documents().get(documentId=DOCUMENT_ID).execute()
    display(f'Title: {result["title"]} - id: {result["documentId"]}')
    return result
  except:
    display(f'FAIL to get {DOCUMENT_ID} SORRY')
# Get Gdoc content blocks in body
def getDocContent(doc):
  try:
    content = []
    for i,z in enumerate(doc['body']['content']):
      content.append(z)
      print(i,z)
    return content
  except:
    display(f'FAIL to get contents for {doc["title"]}')


In [ ]:
 # define client secrets and Google Scopes
CLIENT_SECRETS_FILE = "CRED-DESK.json"
SCOPES = ['https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/documents', 'https://www.googleapis.com/auth/spreadsheets']
# verify credentials run once when initalizing workflow
credentials = getGcreds(CLIENT_SECRETS_FILE,SCOPES)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=244871284916-ieooau3uf8ilc792obrsk33nu933918c.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocuments+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets&state=n2BPzhwgW0oFfobwFVcd2YCUy5K5Gt&prompt=consent&access_type=offline
Enter the authorization code: 4/1AY0e-g7idJdPvMVr4GL-AXi1YIejZjc74WHScNQxs0bDKhEIrZ5ni5ScRPM


In [ ]:
# listFiles in drive function
files = listFiles(50, credentials)

In [ ]:
# Copy Gdoc template for editing with updateDoc function
DOCUMENT_ID = '1XSZRy7p0e75NoxFd_J5JyvCDpRGRwWixYc6BadGHLq8'
docTitle = 'WQAR 2021 DRAFT'
copyID = copyFile(DOCUMENT_ID, docTitle, credentials)

In [ ]:
# fileSearch drive, type ie: pdf,doc, s = search query
s = " and name contains 'water'"
#s = "id contains '1uBK5Sgzjuy7pNdKQ-ANf8pskXdM6K8libKfkpZ1z1lo'"
search = fileSearch('sheet', s, credentials)

'Found 0: Fixing The Broken 2019 Water Data Upload - Bar chart 1 - 1LdSq3iBpLm-QFhnJS-LM213ijHNm9xNUdKtrPLa7VG4'

'Found 1: WATER_DATA_2019_CLEANED - 1uBK5Sgzjuy7pNdKQ-ANf8pskXdM6K8libKfkpZ1z1lo'

'Found 2: NHBP Water Access Landings & Boardwalks - 1rloi8C0PTUo36oYuAtLpppR4_v8Zbinyw1d1An0tcVs'

'Found 3: LINKS FOR WATER DATA - 16aLjD-ThXpFIAU42P8M5TO2-xM5s9ppwh-irrifV6Jc'

In [ ]:
search[0]

{'id': '1LdSq3iBpLm-QFhnJS-LM213ijHNm9xNUdKtrPLa7VG4',
 'name': 'Fixing The Broken 2019 Water Data Upload - Bar chart 1'}

In [ ]:
# Gdoc editing workflow, first define creds, then add params
params = [{ 'replaceAllText': { 'containsText': { 'text': '{{REPORT-DATES}}', 'matchCase': 'true', }, 'replaceText': '2018 - 2020'}},
          { 'replaceAllText': { 'containsText': { 'text': '{{PUBLISH-DATE}}', 'matchCase': 'true', }, 'replaceText': 'DRAFT APRIL 2020'}}]
docResult = updateDoc(copyID, params, credentials)

"Edit Successful for id: 1QtikJsM36O7fOxnzFVJODqqbxVF40r7FMbbEdr-nefU Changes made: {'occurrencesChanged': 1}"

In [ ]:
params = [{'insertInlineImage': {'location': {'index': 1},'uri':'http://drive.google.com/uc?export=view&id=1-m420QsB2yrqoxmqN00GMmI2QV6PBtqS',
        'objectSize': {'height': {'magnitude': 50,'unit': 'PT'},'width': {'magnitude': 50,'unit': 'PT'}}}}]

docResult = updateDoc(copyID, params, credentials)

"Edit Successful for id: 1B6qcFG7N9QE7uWgPqHFusMjg-B_hq4In0GAZfGYjG9s Changes made: {'insertInlineImage': {'objectId': 'kix.p754dj9uugct'}}"

In [ ]:
# Direct link to google drive images!
'http://drive.google.com/uc?export=view&id=1-m420QsB2yrqoxmqN00GMmI2QV6PBtqS'
# http://drive.google.com/uc?export=view&id=YOUR-ID-HERE